In [1]:
from tqdm import tqdm
import pandas as pd
import string
from nltk.corpus import stopwords
import nltk

In [2]:
# Download stopwords if necessary
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/Silja/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Load sentiment dictionaries and speech dataframe
sentiment_df_1 = pd.read_csv('sent_dictionary_1.csv')
sentiment_df_2 = pd.read_csv('sent_dictionary_2.csv')
speeches_df = pd.read_pickle('../data/content_df_features_topic.pkl')

In [5]:
# Prepare German stopwords and process speech content
stop_words = set(stopwords.words('german'))
sentiment_scores_1 = dict(zip(sentiment_df_1['v2'], sentiment_df_1['v3']))
sentiment_scores_2 = dict(zip(sentiment_df_2['feature'].str.strip(), sentiment_df_2['sentiment']))

In [6]:
def preprocess_text(text):
    # Lowercase, remove punctuation, and filter out stopwords
    return [word for word in text.lower().translate(str.maketrans('', '', string.punctuation)).split()
            if word not in stop_words]

In [7]:
# Apply preprocessing with tqdm
tqdm.pandas(desc="Preprocessing speeches")
speeches_df['processed_content'] = speeches_df['speech_content'].progress_apply(preprocess_text)

Preprocessing speeches: 100%|██████████| 969857/969857 [15:35<00:00, 1037.05it/s] 


In [8]:
# Calculate sentiment score with sentiment dictionary
def calculate_sentiment_score(words, sentiment_scores):
    total_score = sum(sentiment_scores.get(word, 0) for word in words if word in sentiment_scores)
    return total_score / len(words) if len(words) > 0 else 0

In [9]:
def calculate_two_sentiment_scores(words, sentiment_scores):
    positive_score = sum(sentiment_scores.get(word, 0) for word in words if word in sentiment_scores and sentiment_scores[word] > 0)
    negative_score = sum(abs(sentiment_scores.get(word, 0)) for word in words if word in sentiment_scores and sentiment_scores[word] < 0)
    
    total_words = len(words)
    positivity_score = positive_score / total_words if total_words > 0 else 0
    negativity_score = negative_score / total_words if total_words > 0 else 0
    
    return positivity_score, negativity_score

In [10]:
tqdm.pandas(desc="Computing sentiment score with dictionary 1")
speeches_df['sentiment_score_1'] = speeches_df['processed_content'].progress_apply(
    lambda words: calculate_sentiment_score(words, sentiment_scores_1)
)

Computing sentiment score with dictionary 1:   0%|          | 0/969857 [00:00<?, ?it/s]

Computing sentiment score with dictionary 1: 100%|██████████| 969857/969857 [02:04<00:00, 7793.10it/s] 


In [11]:
tqdm.pandas(desc="Computing sentiment score with dictionary 2")
speeches_df['sentiment_score_2'] = speeches_df['processed_content'].progress_apply(
    lambda words: calculate_sentiment_score(words, sentiment_scores_2)
)

Computing sentiment score with dictionary 2:   0%|          | 0/969857 [00:00<?, ?it/s]

Computing sentiment score with dictionary 2: 100%|██████████| 969857/969857 [02:24<00:00, 6690.15it/s] 


In [12]:
tqdm.pandas(desc="Computing positivity and negativity scores with dictionary 2.1")

speeches_df[['sentiment_score_2_positivity', 'sentiment_score_2_negativity']] = speeches_df['processed_content'].progress_apply(
    lambda words: pd.Series(calculate_two_sentiment_scores(words, sentiment_scores_2))
)

Computing positivity and negativity scores with dictionary 2.1:   0%|          | 0/969857 [00:00<?, ?it/s]

Computing positivity and negativity scores with dictionary 2.1: 100%|██████████| 969857/969857 [22:44<00:00, 710.83it/s]  


In [13]:
speeches_df.drop(columns=['processed_content'], inplace=True)

In [14]:
speeches_df

,id,electoral_term,session,first_name,document_url,last_name,faction_id,position_short,position_long,politician_id,...,age,gender,tenure,topic,main_category,sub_category,sentiment_score_1,sentiment_score_2,sentiment_score_2_positivity,sentiment_score_2_negativity
0,0,1,2,,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,köhler,5,Presidium of Parliament,präsident,11001150,...,57.0,männlich,0.000000,NaN,NaN,NaN,0.000000,0.047619,0.047619,0.000000
1,1,1,2,,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,arnold,-1,Guest,präsident des bundesrats,-1,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
2,2,1,2,,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,köhler,5,Presidium of Parliament,präsident,11001150,...,57.0,männlich,0.000000,NaN,NaN,NaN,0.133946,0.181818,0.212121,0.030303
3,3,1,2,,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,heuss,-1,Guest,bundespräsident,-1,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,1.000000,1.000000,0.000000
4,4,1,2,,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,köhler,5,Presidium of Parliament,präsident,11001150,...,57.0,männlich,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969852,1094071,20,201,thomas,https://dip21.bundestag.de/dip21/btp/20/20201.pdf,lutze,7,Member of Parliament,None,11004103,...,55.0,männlich,14.734247,Arbeit: Mindestlohn,Work,Mindestlohn,0.132517,0.063492,0.154762,0.091270
969853,1094072,20,201,bärbel,https://dip21.bundestag.de/dip21/btp/20/20201.pdf,bas,-1,Presidium of Parliament,Präsidentin,11004006,...,56.0,weiblich,14.830137,No Topic,no topic,None,0.000000,0.307692,0.307692,0.000000
969854,1094073,20,201,wilfried,https://dip21.bundestag.de/dip21/btp/20/20201.pdf,oellers,5,Member of Parliament,None,11004365,...,49.0,männlich,10.690411,Sozialpolitik,Social Policy,None,0.135320,0.055385,0.132308,0.076923
969855,1094074,20,201,bärbel,https://dip21.bundestag.de/dip21/btp/20/20201.pdf,bas,-1,Presidium of Parliament,Präsidentin,11004006,...,56.0,weiblich,14.830137,No Topic,no topic,None,0.000000,0.000000,0.142857,0.142857


In [15]:
# Save results
speeches_df.to_pickle('content_df_features_sentiment.pkl')

In [16]:
speeches_df.to_csv('content_df_features_sentiment.csv')